In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Train_SU63ISt.csv')
df

In [ ]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [ ]:
df.set_index('Datetime', inplace=True)
df.drop('ID', axis=1, inplace=True)


In [ ]:
df['Count'].plot()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df['Count'], model='additive')
result.plot()
plt.show()

In [ ]:
df['Count'].describe()

In [ ]:
df['Count'].plot(kind='hist', bins=30)
plt.title('Histogram of Traffic Counts')
plt.xlabel('Count')
plt.show()

df['Count'].plot(kind='box')
plt.title('Boxplot of Traffic Counts')
plt.show()

In [ ]:
# Convert the index to datetime if it's not already
df.index = pd.to_datetime(df.index)

df['day_of_week'] = df.index.dayofweek  # Monday=0, Sunday=6
df['hour'] = df.index.hour
df['month'] = df.index.month
df['day_of_month'] = df.index.day
df

In [ ]:
df.columns

In [ ]:
import seaborn as sns

# Assuming 'day_of_week' and 'hour' features are created
correlation_matrix = df[['Count', 'day_of_week', 'hour', 'day_of_month', 'month']].corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
# Group the data by 'month' and sum the 'Count' for each month
monthly_counts = df.groupby('month')['Count'].mean()

# Plot the total 'Count' for each month as a line or bar chart
monthly_counts.plot(kind='bar')  # You can change 'bar' to 'line' if you prefer a line plot

plt.title('Mean Count by Month')
plt.xlabel('Month')
plt.ylabel('Total Count')
plt.xticks(ticks=range(len(monthly_counts)), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Group the data by 'month' and sum the 'Count' for each month
monthly_counts = df.groupby('month')['Count'].sum()

# Plot the total 'Count' for each month as a line or bar chart
monthly_counts.plot(kind='bar')  # You can change 'bar' to 'line' if you prefer a line plot

plt.title('Total Count by Month')
plt.xlabel('Month')
plt.ylabel('Total Count')
plt.xticks(ticks=range(len(monthly_counts)), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
plt.show()

In [ ]:
# Group the data by 'month' and sum the 'Count' for each month
monthly_counts = df.groupby('day_of_week')['Count'].mean()

# Plot the total 'Count' for each month as a line or bar chart
monthly_counts.plot(kind='bar')  # You can change 'bar' to 'line' if you prefer a line plot

plt.title('Mean Count by DoW')
plt.xlabel('Day of Week')
plt.ylabel('Total Count')
plt.xticks(ticks=range(len(monthly_counts)), labels=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], rotation=45)
plt.show()

In [ ]:
df['hour'] = df.index.hour

df['month'] = df.index.month
df['year'] = df.index.year
df.columns

In [ ]:
# First, resample and sum the 'Count' data to get a Series
daily_counts = df['Count'].resample('D').sum()

# Then, create the seasonal lag by shifting the Series
count_seasonal_lag = daily_counts.shift(365)

# If you want to combine them into a new DataFrame:
daily_df = pd.DataFrame({'Count': daily_counts, 'Count_seasonal_lag': count_seasonal_lag})

daily_df

In [ ]:
from pmdarima import auto_arima

model = auto_arima(daily_df['Count'], seasonal=True, m=365, trace=True, 
                   error_action='ignore', suppress_warnings=True)

In [9]:
future_df = pd.read_csv('Test_0qrQsBZ.csv')
future_df

ID          Datetime
0     18288  26-09-2014 00:00
1     18289  26-09-2014 01:00
2     18290  26-09-2014 02:00
3     18291  26-09-2014 03:00
4     18292  26-09-2014 04:00
...     ...               ...
5107  23395  26-04-2015 19:00
5108  23396  26-04-2015 20:00
5109  23397  26-04-2015 21:00
5110  23398  26-04-2015 22:00
5111  23399  26-04-2015 23:00

[5112 rows x 2 columns]

In [10]:
future_df.drop('ID', axis=1, inplace=True)

In [12]:
future_df['Datetime']

0       26-09-2014 00:00
1       26-09-2014 01:00
2       26-09-2014 02:00
3       26-09-2014 03:00
4       26-09-2014 04:00
              ...       
5107    26-04-2015 19:00
5108    26-04-2015 20:00
5109    26-04-2015 21:00
5110    26-04-2015 22:00
5111    26-04-2015 23:00
Name: Datetime, Length: 5112, dtype: object

In [14]:
# Make predictions
n_periods = len(future_df['Datetime'])
forecasted_counts = model.predict(n_periods=n_periods)

NameError: name 'model' is not defined

In [ ]:
# Create a DataFrame with the future dates and the forecasted counts
future_df = pd.DataFrame({'Datetime': future_dates, 'Predicted_Count': forecasted_counts})
future_df.set_index('Datetime', inplace=True)

In [ ]:
rom sklearn.metrics import mean_squared_error
from math import sqrt

# If you have actual test data to compare against
test_actual = ...  # your actual observed counts

# Compute RMSE
rmse = sqrt(mean_squared_error(test_actual, future_forecasts[:len(test_actual)]))
print(f'The RMSE is: {rmse}')

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Generate predictions
predictions = model.predict(n_periods=len(test))

# Calculate RMSE
rmse = sqrt(mean_squared_error(test['Count'], predictions))